# Notebook 1: Data pre-processing and management.

This is the first notebook in the final project for Machine Learning. 

The objective of this preprocessing stage is to construct a structured dataset of fixed-size spectrograms derived from annotated seal calls. These spectrograms will serve as the input features for subsequent machine learning analyses aimed at discriminating between different call types. An additional “no-call” class is also generated to represent background audio segments without vocalisations.

All preprocessing steps are implemented within a single Jupyter notebook to ensure transparency, reproducibility, and ease of inspection.

## Imports

In [ ]:
# https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html
import matplotlib.pyplot as plt
# https://docs.scipy.org/doc/scipy/tutorial/signal.html
from scipy import signal
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.html
from scipy.io import wavfile
# https://numpy.org/doc/stable/
import numpy as np
# https://pandas.pydata.org/docs/
import pandas as pd
# https://matplotlib.org/stable/api/colors_api.html#matplotlib.colors.LogNorm
from matplotlib.colors import LogNorm
# to enable zooming in matplotlib in notebook
#%matplotlib notebook 
#conda install -c conda-forge ipympl # or pip install ipympl #This is to allow zooming in the notebook
# %matplotlib ipympl

Note the path to the data files. The data is not stored in the repository (as it is too large and also private to ATU's MFRC). Instead it is stored in the one drive on the desktop. 
Each recording consists of:
A .wav audio file containing the continuous acoustic recording

A corresponding annotation file (.Table.1.selections.txt) specifying call start time, call end time, frequency bounds, call type (e.g. Rupe A, Rupe B, Gutteral and Moan)

These annotations were produced using acoustic analysis software and are treated as ground truth.

In [ ]:
#File index
file =  r'C:\Users\grace\OneDrive\Desktop\GreySealData\Rupes A and B\5713.210825190002'  #from PPT at time 892-896 (Rupe B)
#file = r'C:\Users\grace\OneDrive\Desktop\GreySealData\Guttural rupe\\5711.211013040024'
#file = r'C:\Users\grace\OneDrive\Desktop\GreySealData\\Rupes A and B\\5713.210825190002'
#file = r'C:\Users\grace\OneDrive\Desktop\GreySealData\\Moan\\5713.210902110002'  #from PPT at time 212 seconds

#Read the 2 files
sample_rate, samples = wavfile.read(file+'.wav')
annot_file_path = file +'.Table.1.selections.txt'

#Read the file into a DataFrame
df = pd.read_csv(file +'.Table.1.selections.txt', sep='\t')

#Display the first few rows of the DataFrame
print(df.head())
print(f'Sample rate: {sample_rate} Hz')

## Spectogram Computation

A short time Fourier transform-based spectogram is computed using parameters selected to balance time and frequency resolution while capturing the frequency range of seal vocalisations

https://en.wikipedia.org/wiki/Short-time_Fourier_transform

In [ ]:
# Calculate the Spectrogram
frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate, nperseg=2456, nfft=4096, noverlap=1228, window='hann')       #Try nfft=8192 if computer permits

## Frequency band selection
With the resultsing spectrogram, trim all the tiny values so that log scale displays correctly. Also, all of the relevant info is below 1KHz so trim the data to only display sub 1-KHz frequencies. Restricting the frequency range reduces dimensionality and removes irrelevant noise.

In [ ]:
# Calculate the Spectrogram
frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate, nperseg=2456, nfft=4096, noverlap=1228, window='hann')       #Try nfft=8192 if computer permits

## Visual Validation with Annotations.
Define a function that will overlay the annotated rectangles onto the spectrogram (different colours for each call)

In [ ]:
def overlay_annotations(ax, df, annotation_colors):
    #Track labels to ensure they are added only once in the legend
    added_labels = set()

    for _, row in df.iterrows():
        start_time = row['Begin Time (s)']
        end_time = row['End Time (s)']
        low_freq = row['Low Freq (Hz)']
        high_freq = row['High Freq (Hz)']
        annotation = row['Annotation']

        #Skip if the annotation is not in the defined colors
        if annotation not in annotation_colors:
            continue

        #Draw rectangles
        ax.add_patch(
            plt.Rectangle(
                (start_time, low_freq),  #Bottom Left corner
                end_time - start_time,  #Width (time)
                high_freq - low_freq,  #Height (frequency)
                edgecolor=annotation_colors[annotation],
                facecolor='none',
                linewidth=2,
                label=annotation if annotation not in added_labels else None  #Add label once
            )
        )
        added_labels.add(annotation)  #Mark label as added

    ax.legend(loc='upper right')     #Add legend

A function below that updates the colormap when you zoom in on a particular region - so that the max and min values are always visible.

In [ ]:
def update_colormap(event):
    #Get current view limits
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    #Find indices corresponding to the current view limits
    x_indices = np.where((times >= xlim[0]) & (times <= xlim[1]))[0]
    y_indices = np.where((frequencies >= ylim[0]) & (frequencies <= ylim[1]))[0]

    #Handle cases where no data is visible
    if len(x_indices) == 0 or len(y_indices) == 0:
        return

    #Extract the visible data
    data_visible = spectrogram[np.ix_(y_indices, x_indices)]
    #data_visible = np.log(spectrogram)[np.ix_(y_indices, x_indices)]

    #Compute new color limits
    vmin = np.nanmin(data_visible)
    vmax = np.nanmax(data_visible)

    #Update the color limits of the pcolormesh
    pc.set_clim(vmin=vmin, vmax=vmax)
    
    #Update the colorbar to reflect the new color limits
    cbar.update_normal(pc)

    #Redraw the figure
    plt.draw()
    

Plot the spectrogram and annotations

In [ ]:
#Define colors for annotations
annotation_colors = {
    "Rupe A": "red",
    "Rupe B": "green",
    "Growl B": "yellow",
    "Rupe C" : "purple",
    "Moan": "pink",
    "G rupe" : "blue"
}

The following cell is the quality assurance/inspection tool

In [ ]:

pc = plt.pcolormesh(times, frequencies, spectrogram, norm=LogNorm(), cmap='Spectral_r')
#pc = plt.pcolormesh(times, frequencies, np.log(spectrogram))
cbar = plt.colorbar(pc)
#plt.imshow(spectrogram)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')

#Get the current axes
ax = plt.gca()
overlay_annotations(ax, df, annotation_colors)

#Connect the update function to the axes limit change events
ax.callbacks.connect('xlim_changed', update_colormap)
ax.callbacks.connect('ylim_changed', update_colormap)

plt.show()

Extract out a portion of interest from the spectrogram. This is the manual Sub-Spectogram Extraction (Ground Truth Validation). It confirms shape, orientation and frequency coverage.

#Define time and frequency limits
time_start, time_end = 891, 897  # Time range in seconds
freq_start, freq_end = 20, 600  # Frequency range in Hz

#Find indices for the time range
time_indices = np.where((times >= time_start) & (times <= time_end))[0]

#Find indices for the frequency range
freq_indices = np.where((frequencies >= freq_start) & (frequencies <= freq_end))[0]

#Extract the portion of the spectrogram
spectrogram_sub = spectrogram[freq_indices][:, time_indices]
frequencies_sub = frequencies[freq_indices]
times_sub = times[time_indices]
print(spectrogram_sub.shape)

#Plot the original and sub-portion spectrograms
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.pcolormesh(times, frequencies, spectrogram, norm=LogNorm(), cmap='Spectral_r')
plt.title('Original Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.colorbar(label='Power [dB]')

plt.subplot(2, 1, 2)
plt.pcolormesh(times_sub, frequencies_sub, spectrogram_sub, norm=LogNorm(), cmap='Spectral_r')
plt.title('Extracted Portion of Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.colorbar(label='Power [dB]')

plt.tight_layout()
plt.show()


The above cell confirms extraction is correct.

Extract a single call and check the size of the spectrogram array.

In [ ]:
#Define time and frequency limits
time_start, time_end = 892.5, 893.2  # Time range in seconds
freq_start, freq_end = 20, 600  # Frequency range in Hz

#Find indices for the time range
time_indices = np.where((times >= time_start) & (times <= time_end))[0]

#Find indices for the frequency range
freq_indices = np.where((frequencies >= freq_start) & (frequencies <= freq_end))[0]

#Extract the portion of the spectrogram
spectrogram_sub = spectrogram[freq_indices][:, time_indices]
frequencies_sub = frequencies[freq_indices]
times_sub = times[time_indices]
print("Spectrogram size: ", spectrogram_sub.shape)

#Plot the original and sub-portion spectrograms
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.pcolormesh(times, frequencies, spectrogram, norm=LogNorm(), cmap='Spectral_r')
plt.title('Original Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.colorbar(label='Power [dB]')

plt.subplot(2, 1, 2)
plt.pcolormesh(times_sub, frequencies_sub, spectrogram_sub, norm=LogNorm(), cmap='Spectral_r')
plt.title('Extracted Portion of Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.colorbar(label='Power [dB]')

plt.tight_layout()
plt.show()

## printing the Spectogram Array

https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html

## Determining the baseline spectogram size.

This is to ensure consistancy across all samples. All the spectograms must have the same dimentions. Identification of the longest call duration and the maximum frequency bin count after frequency trimming

In [ ]:
df['duration'] = df['End Time (s)'] - df['Begin Time (s)']
max_duration = df['duration'].max()
max_duration

## Extracting Call-Centred Spectograms.

Each call is extracted by centering in a fixed-length window on the midpoint of the annoted call. ie the call is in the middle of the sample.

In [ ]:
def extract_call_spectrogram(center_time, duration):
    start_sample = int((center_time - duration / 2) * sample_rate)
    end_sample = int((center_time + duration / 2) * sample_rate)

    segment = samples[start_sample:end_sample]

    f, t, Sxx = signal.spectrogram(
        segment,
        fs=sample_rate,
        nperseg=2456,
        nfft=4096,
        noverlap=1228,
        window='hann'
    )

    Sxx[Sxx < 0.001] = 0.001
    Sxx = Sxx[freq_indices, :]

    # Log scaling for ML
    return np.log10(Sxx + 1e-10)
    return Sxx

## Extracting and saving call Spectograms

Saved as raw NumPy arrays

In [ ]:
# List of files to process
files = [
    r'C:\Users\grace\OneDrive\Desktop\GreySealData\Rupes A and B\5713.210825190002',
    r'C:\Users\grace\OneDrive\Desktop\GreySealData\Guttural rupe\5711.211013040024',
    r'C:\Users\grace\OneDrive\Desktop\GreySealData\Rupes A and B\5713.210825190002',
    r'C:\Users\grace\OneDrive\Desktop\GreySealData\Moan\5713.210902110002'
 ]

# Initialize combined lists for spectrograms and metadata
combined_spectrograms = []
combined_metadata = []

for file in files:
    # Read the audio file and annotation file
    sample_rate, samples = wavfile.read(file + '.wav')
    df = pd.read_csv(file + '.Table.1.selections.txt', sep='\t')

    # Process each annotation in the file
    for _, row in df.iterrows():
        center_time = (row['Begin Time (s)'] + row['End Time (s)']) / 2
        spec = extract_call_spectrogram(center_time, max_duration)

        combined_spectrograms.append(spec)
        combined_metadata.append({
            "call_type": row['Annotation'],
            "center_time": center_time,
            "duration": max_duration
        })

# Save the combined results
np.save("call_spectrograms.npy", np.array(combined_spectrograms))
pd.DataFrame(combined_metadata).to_csv("call_metadata.csv", index=False)

## "No Call" spectogram.

To provide a negative class, spectrograms are extracted from unannotated regions. These samples use the same duration and frequency range as call spectrograms.

In [ ]:
# https://numpy.org/doc/2.3/reference/generated/numpy.linspace.html
no_call_times = np.linspace(
    max_duration,
    times[-1] - max_duration,
    10
)

no_call_specs = []

for t in no_call_times:
    no_call_specs.append(extract_call_spectrogram(t, max_duration))

np.save("no_call_spectrograms.npy", np.array(no_call_specs))

no_call_times = np.linspace(
    max_duration,
    times[-1] - max_duration,
    10
)

no_call_specs = []

for t in no_call_times:
    no_call_specs.append(extract_call_spectrogram(t, max_duration))

np.save("no_call_spectrograms.npy", np.array(no_call_specs))


In [ ]:
print(spectrogram_sub)


This is the 2D numpy array that the ML model will see. Each row = frequency bin and each column = time bin. The values are the spectral power (log-scaled via plotting)

## Log scale before saving for ML

In [ ]:
# https://www.kaggle.com/code/theoviel/spectrogram-generation
spectrogram_sub_ml = np.log10(spectrogram_sub + 1e-10)
print(spectrogram_sub_ml)

Log scaled spectograms are used in Machine learning as input features in audio classification tasks, including music analysis, speech processing and bioacoustics, which is what we have here. Studies show that models trained on logarithmic or decibel-scaled spectograms consistently outperform those trained on linear-scaled representations.

By applying this transformation prior to saving the data, the resulting 2D arrays become directly suitable for classical machine learning models (e.g., SVMs, random forests) as well as convolutional neural networks.

## Summary of this notebook

This is the preprocessing pipeline. It converts large raw data from the <i>'Grey Seal Project'</i> into fixed sized spectograms while preserving the spectral and temporal consistency. It produces both call and no call examples and stores data in a format usable for Machine Learning ie numerical tensor form not as images. Call examples correspond to annotated grey seal vocalisations (e.g. Rupe A, Rupe B, Moan), as defined in the provided annotation files. No-call examples are extracted from unannotated regions of the recordings and represent background acoustic conditions. 

# END